<a href="https://colab.research.google.com/github/vadhri/hpc-notebook/blob/main/mpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install mpi4py
!apt-get install -y openmpi-bin


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-bin set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
!mpiexec --version

mpiexec (OpenRTE) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/


In [13]:
%%writefile hello.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
procs = comm.Get_size()
rank = comm.Get_rank()
if rank == 0:
  print(f"Number of processes: {procs}") ## will be printed by each process.(or each core / machine.)

print(f"Message from processes: {procs} Rank : {rank}")


Overwriting send_recv.py


In [37]:
!sudo mpirun  --allow-run-as-root --use-hwthread-cpus -np 2 python hello.py

Number of processes: 2
Message from processes: 2 Rank : 0
Message from processes: 2 Rank : 1


In [36]:
!lscpu | grep 'CPU(s)'


CPU(s):                               2
On-line CPU(s) list:                  0,1
NUMA node0 CPU(s):                    0,1


In [40]:
!echo "localhost slots=4" > my_hostfile
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python send_recv.py

Message from processes: 4 Rank : 2
Number of processes: 4
Message from processes: 4 Rank : 0
Message from processes: 4 Rank : 3
Message from processes: 4 Rank : 1


### Single send-recv

In [42]:
%%writefile send_recv.py

from mpi4py import MPI

comm = MPI.COMM_WORLD  # Get the communicator
rank = comm.Get_rank()  # Get the rank of the current process

if rank == 0:
    data = "Hello from Process 0"
    comm.send(data, dest=1)  # Send data to process 1
    print(f"Process {rank} sent: {data}")

elif rank == 1:
    received_data = comm.recv(source=0)  # Receive data from process 0
    print(f"Process {rank} received: {received_data}")


Overwriting send_recv.py


In [43]:
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python send_recv.py

Process 0 sent: Hello from Process 0
Process 1 received: Hello from Process 0


### Bidirectional send-receive

In [45]:
%%writefile send_recv_bidirectional.py
from mpi4py import MPI

comm = MPI.COMM_WORLD  # Get the communicator
rank = comm.Get_rank()  # Get the rank of the current process

if rank == 0:
    data_to_send = "Hello from Process 0"
    comm.send(data_to_send, dest=1)  # Send to Process 1
    received_data = comm.recv(source=1)  # Receive from Process 1
    print(f"Process {rank} sent: {data_to_send}")
    print(f"Process {rank} received: {received_data}")

elif rank == 1:
    data_to_send = "Hello from Process 1"
    received_data = comm.recv(source=0)  # Receive from Process 0
    comm.send(data_to_send, dest=0)  # Send to Process 0
    print(f"Process {rank} received: {received_data}")
    print(f"Process {rank} sent: {data_to_send}")


Writing send_recv_bidirectional.py


In [47]:
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python send_recv_bidirectional.py

Process 1 received: Hello from Process 0
Process 1 sent: Hello from Process 1
Process 0 sent: Hello from Process 0
Process 0 received: Hello from Process 1


In [49]:
%%writefile bidirec_square_numbers.py
from mpi4py import MPI
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

num_iterations = 5  # Number of times we send/receive numbers

if rank == 0:  # Sender process
    for i in range(1, num_iterations + 1):
        print(f"Process {rank} sending: {i}")
        comm.send(i, dest=1)  # Send number to Process 1
        squared_value = comm.recv(source=1)  # Receive squared value
        print(f"Process {rank} received squared: {squared_value}")
        time.sleep(1)  # Just to simulate processing delay

elif rank == 1:  # Receiver process
    for _ in range(num_iterations):
        received_number = comm.recv(source=0)  # Receive number from Process 0
        squared_number = received_number ** 2  # Square the number
        comm.send(squared_number, dest=0)  # Send squared number back



Writing bidirec_square_numbers.py


In [50]:
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python bidirec_square_numbers.py

Process 0 sending: 1
Process 0 received squared: 1
Process 0 sending: 2
Process 0 received squared: 4
Process 0 sending: 3
Process 0 received squared: 9
Process 0 sending: 4
Process 0 received squared: 16
Process 0 sending: 5
Process 0 received squared: 25


### Process sync with shared memory

In [52]:
%%writefile bidirec_shared_memory.py

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 5  # Number of iterations
array_size = 4  # Size of shared array

# Create a shared memory buffer
win = MPI.Win.Allocate_shared(array_size * np.dtype('i').itemsize, np.dtype('i').itemsize, comm=comm)

# Get shared memory array reference
buf, itemsize = win.Shared_query(0)  # Rank 0 creates the memory
shared_array = np.ndarray(buffer=buf, dtype='i', shape=(array_size,))

# Create a flag in shared memory
flag_win = MPI.Win.Allocate_shared(np.dtype('i').itemsize, np.dtype('i').itemsize, comm=comm)
flag_buf, _ = flag_win.Shared_query(0)
flag = np.ndarray(buffer=flag_buf, dtype='i', shape=(1,))

if rank == 0:
    for iter in range(N):
        # Initialize array
        shared_array[:] = np.arange(1, array_size + 1) * (iter + 1)
        print(f"Process 0: Initialized array: {shared_array}")

        # Signal process 1 to start processing
        flag[0] = 1
        flag_win.Sync()

        # Wait for process 1 to finish
        while flag[0] != 2:
            flag_win.Sync()
            time.sleep(0.01)

        # Sum the modified array
        total = np.sum(shared_array)
        print(f"Process 0: Sum after modification: {total}")

        # Reset flag
        flag[0] = 0
        flag_win.Sync()

elif rank == 1:
    for _ in range(N):
        # Wait for signal from process 0
        while flag[0] != 1:
            flag_win.Sync()
            time.sleep(0.01)

        # Multiply values by 2
        shared_array[:] *= 2
        print(f"Process 1: Modified array: {shared_array}")

        # Signal process 0 that modification is done
        flag[0] = 2
        flag_win.Sync()


Writing bidirec_shared_memory.py


In [53]:
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python bidirec_shared_memory.py

Process 0: Initialized array: [1 2 3 4]
Process 1: Modified array: [2 4 6 8]
Process 0: Sum after modification: 20
Process 0: Initialized array: [2 4 6 8]
Process 1: Modified array: [ 4  8 12 16]
Process 0: Sum after modification: 40
Process 0: Initialized array: [ 3  6  9 12]
Process 1: Modified array: [ 6 12 18 24]
Process 0: Sum after modification: 60
Process 0: Initialized array: [ 4  8 12 16]
Process 1: Modified array: [ 8 16 24 32]
Process 0: Sum after modification: 80
Process 0: Initialized array: [ 5 10 15 20]
Process 1: Modified array: [10 20 30 40]
Process 0: Sum after modification: 100
